In [1]:
import random
import math

In [2]:
# We have implemented the events as a class which holds 3 information:
# event type, occurence time and the id of related patient (enumerated by arriving order)
# events related to end of triages are called departure1 and departure2 which corresponds the nurse1 and nurse2
class Event:
 
    def __init__(self, type, time, id):
        self.type = type
        self.time = time
        self.id=id
 

In [3]:
random.seed(4038800291)

In [4]:
num_of_events=0

num_of_avai_nurses=2
sick_people=0 # sick people is the number of people that are arrived but hasn's healed yet
nurse1=True # availability of first nurse
nurse2=True # availability of second nurse
nurse1_start_time=0 # the time nurse 1 has started his/her current triage (or triages if no break between)
nurse2_start_time=0 # the time nurse 2 has started his/her current triage (or triages if no break between)
nurse1_work = 0 # total amount of time spent by nurse 1 on triages
nurse2_work = 0 # total amount of time spent by nurse 2 on triages

num_of_beds=5 # total number of beds 
last_bed_num_change_time = 0 # the time which number of available beds changed last time
total_bed_occupation = 0 # this variable corresponds to the area under the number of available beds over time graph
num_of_avai_beds=5 # number of beds that are currently available

num_of_patients_queue=0 # number of people that are in the triage queue at current time
current_event=None 
current_time=0 # time of current event
future_event_list=[]
queue=[] # triage queue

total_arrivals=0 # number of total arrivals (including people that are healed)
rejected_patient=0 # number of patients who are rejected due to bed unavailability
accepted_patient=0 # number of patients that are accepted to beds
stable_patient=0 # number of patients whom the conditions' are stable

empty_bed_time=0 # total amount of time that at least one bed is available
empty_nurse_time=0 # total amount of time that at least one nurse is available
both_empty_time=0 # total amount of time that at least one nurse and bed is available

arrival_times=[] # arrival times of each patient (by their arriving order)
total_healing_time=0 # total amount of healing time spent by all patients
total_healed_patient=0 # number of healed patients

In [5]:
def generate_interarrival():
    randInt=random.random()
    return math.log(1-randInt)*-1
# probability of event occurrence less then time t is 1-e^(-lambada*t)
# so RN=1-e^(-lambada*t)
# e^(-lambada*t)=1-RN
# ln(1-RN)=-lambda*t
# (ln(1-RN))/lambda=t
# lambda is 1 here

In [6]:
def generate_nurse_service_time():
    randInt=random.random()
    return (math.log(1-randInt)*-1)/0.666666667
# lambda is 0.66 here

In [7]:
def generate_hospital_healing_time():
    randInt=random.random()
    return (math.log(1-randInt)*-1)/0.1875
# lambda is 0.1875 here

In [8]:
def generate_home_healing_time(input1):
    randInt=random.random()
    if input1=='s':
        return (math.log(1-randInt)*-1)/0.16
    # lambda is 0.16 here
    elif input1=='c':
        randInt2=random.random()
        return ((math.log(1-randInt)*-1)/0.1875)*((randInt2/2)+1.25)
    # lambda is 0.1875 here, alpha is uniform random value between 1.25 and 1.75

In [9]:
def generate_home_probability():
    randInt=random.random()
    return randInt<=0.25

In [10]:
def arrival():
    global num_of_avai_nurses
    global num_of_patients_queue
    global nurse1
    global nurse2
    global future_event_list
    global current_time
    global nurse1_start_time
    global nurse2_start_time
    global queue
    global total_arrivals
    
    # arrival time of next arrival is calculated and added to the queue
    arrival_times.append(current_time)
    next_arrival_time=current_time+generate_interarrival()
    new_event1=Event("arrival", next_arrival_time, total_arrivals+1)
    future_event_list.append(new_event1)
    
    if num_of_avai_nurses==0:
        # if no nurses are available to start triage patient is added to the queue
        num_of_patients_queue+=1
        queue.append(total_arrivals)
    else:
        # if there are nurses available triage event is added (nurse 1 has priority to start)
        num_of_avai_nurses-=1
        if nurse1==True:
            nurse1=False # availability is changed to false
            
            nurse1_start_time = current_time
            time=current_time+generate_nurse_service_time()
            new_event=Event("departure1", time, total_arrivals)
            future_event_list.append(new_event)
        else:
            nurse2=False
            
            nurse2_start_time = current_time
            time=current_time+generate_nurse_service_time()
            new_event=Event("departure2", time, total_arrivals)
            future_event_list.append(new_event)


In [11]:
def departure_triage():
    global num_of_patients_queue
    global num_of_avai_nurses
    global nurse1
    global nurse2
    global future_event_list
    global current_time
    global current_event
    global num_of_avai_beds
    global total_bed_occupation
    global last_bed_num_change_time
    global nurse1_start_time
    global nurse2_start_time
    global nurse1_work
    global nurse2_work
    global rejected_patient
    global accepted_patient
    global stable_patient
    
    # based on the results of triage necessary events are added to fel
    if generate_home_probability(): # if patient is stable, he/she is sent home
        stable_patient+=1
        healing_time=generate_home_healing_time('s')+current_time
        new_event=Event("home", healing_time, current_event.id)
        future_event_list.append(new_event)
    elif num_of_avai_beds==0: # if critical but no beds still sent home
        rejected_patient += 1
        healing_time=generate_home_healing_time('c')+current_time
        new_event=Event("home", healing_time, current_event.id)
        future_event_list.append(new_event)
    else: # if critical and there are available beds, gets treatment at hospital
        accepted_patient += 1
        
        # keeping track of area under the number of avalibale beds graph  
        total_bed_occupation += (5-num_of_avai_beds) * (current_time - last_bed_num_change_time)
        last_bed_num_change_time = current_time
        num_of_avai_beds-=1
        
        healing_time=generate_hospital_healing_time()+current_time
        new_event=Event("hospital", healing_time, current_event.id)
        future_event_list.append(new_event)
        
    if num_of_patients_queue==0:
        # if there are no patients waiting then nurse stops working and amount of work is added to the total
        num_of_avai_nurses+=1
        if current_event.type=="departure1":
            nurse1=True
            nurse1_work += current_time - nurse1_start_time
        else:
            nurse2=True
            nurse2_work += current_time - nurse2_start_time
    else:
        # if there are patients waiting they start triage and necessary event is added 
        num_of_patients_queue-=1
        queue_head = queue.pop(0)
        time=current_time+generate_nurse_service_time()
        new_event=Event(current_event.type, time, queue_head)
        future_event_list.append(new_event)


In [12]:
def treated_at_hospital():
    global num_of_avai_beds
    global total_bed_occupation
    global num_of_avai_beds
    global current_time
    global last_bed_num_change_time
    global total_healed_patient
    global arrival_times
    global total_healing_time
    global current_event
    
    # keeping track of total amount of time all patients spent healing
    total_healed_patient+=1
    patient = current_event.id
    total_healing_time += current_time - arrival_times[patient]
    
    # keeping track of area under the number of avalibale beds graph  
    total_bed_occupation += (5-num_of_avai_beds) * (current_time - last_bed_num_change_time)
    last_bed_num_change_time = current_time
    num_of_avai_beds+=1
    

In [13]:
def treated_at_home():
    global current_event
    global arrival_times
    global total_healing_time
    global total_healed_patient
    
    # keeping track of total amount of time all patients spent healing
    total_healed_patient+=1
    patient = current_event.id
    total_healing_time += current_time - arrival_times[patient]
    

In [14]:
def advance_time():
    global current_time
    global future_event_list
    global num_of_avai_beds
    global num_of_avai_nurses
    global empty_bed_time
    global empty_nurse_time
    global both_empty_time
    
    # the event with the smallest occurence time is selected as next event
    # and moved to the first element of the list
    next_event = min(future_event_list, key=lambda event: event.time)
    index_of_event=future_event_list.index(next_event)
    future_event_list[0],future_event_list[index_of_event]=future_event_list[index_of_event],future_event_list[0]
    
    # current time is set to the next event while last event's time is set to last_time to help
    # calculating the total amount of times that the beds or nurses are available
    last_time = current_time
    current_time=next_event.time
    
    if(num_of_avai_beds>0):
        empty_bed_time+=current_time-last_time
        if(num_of_avai_nurses>0):
            both_empty_time+=current_time-last_time
            empty_nurse_time+=current_time-last_time
    elif num_of_avai_nurses>0:
        empty_nurse_time+=current_time-last_time

In [15]:
def execute_event():
    global current_event
    global total_arrivals
    global sick_people
    global stable_patient
    global num_of_avai_beds
    global num_of_avai_nurses
    global num_of_patients_queue
    global num_of_events
    global future_event_list
    global current_time
    global nurse1
    global nurse2
    
    num_of_events+=1
    # We are getting the nearest event from our FEL and calling the necessary function according to its type
    current_event = future_event_list.pop(0)
    typeOfEvent = current_event.type
    #print(typeOfEvent, current_event.time)
    if typeOfEvent == "arrival":
        arrival()
        total_arrivals+=1
        sick_people+=1
    elif typeOfEvent == "departure1" or typeOfEvent == "departure2":
        departure_triage()
    elif typeOfEvent == "hospital":
        treated_at_hospital()
        sick_people-=1
    else:
        treated_at_home()
        sick_people-=1
    
    #print(sick_people)
    #print(7-num_of_avai_nurses-num_of_avai_beds+num_of_patients_queue) #number of people in hospital

In [ ]:
# This cell is used for first starting condition of being completely empty at the start of simulation

# arrival event of first patient is created by hand
next_arrival_time=current_time+generate_interarrival()
new_event=Event("arrival", next_arrival_time, total_arrivals)
future_event_list.append(new_event)

#while (total_healed_patient) < 20:
#while (total_healed_patient) < 200:
while (total_healed_patient) < 1000:
    advance_time()
    execute_event()


total_bed_occupation += (5-num_of_avai_beds) * (current_time - last_bed_num_change_time)

# since the time nurses have been spending on triage is calculated after finishing them at the end of 
# the simulation working time of nurses are added by hand in case they are still working
if nurse1 == False: 
    nurse1_work += current_time - nurse1_start_time
if nurse2 == False: 
    nurse2_work += current_time - nurse2_start_time

In [ ]:
# This cell is used for second starting condition of being half empty at the start of simulation

# events related to departures and healing of patients that are already at the system are created by hand
num_of_avai_nurses-=1
nurse1=False
total_arrivals=3
sick_people=3
arrival_times.append(0)
arrival_times.append(0)
arrival_times.append(0)
time=current_time+generate_nurse_service_time()
new_event=Event("departure1", time, 0)
future_event_list.append(new_event)

num_of_avai_beds-=2
healing_time=generate_hospital_healing_time()+current_time
new_event=Event("hospital", healing_time, 1)
future_event_list.append(new_event)

healing_time=generate_hospital_healing_time()+current_time
new_event=Event("hospital", healing_time, 2)
future_event_list.append(new_event)
    
# arrival event of first patient is created by hand
next_arrival_time=current_time+generate_interarrival()
new_event=Event("arrival", next_arrival_time, total_arrivals)
future_event_list.append(new_event)

#while (total_healed_patient) < 20:
#while (total_healed_patient) < 200:
while (total_healed_patient) < 1000:
    advance_time()
    execute_event()


total_bed_occupation += (5-num_of_avai_beds) * (current_time - last_bed_num_change_time)

# since the time nurses have been spending on triage is calculated after finishing them at the end of 
# the simulation working time of nurses are added by hand in case they are still working
if nurse1 == False: 
    nurse1_work += current_time - nurse1_start_time
if nurse2 == False: 
    nurse2_work += current_time - nurse2_start_time

In [16]:
# This cell is used for third starting condition of being completely full at the start of simulation

# events related to departures and healing of patients that are already at the system are created by hand
num_of_avai_nurses-=2
nurse1=False
nurse2=False
total_arrivals=7
sick_people=7
for i in range(7):
    arrival_times.append(0)
time=current_time+generate_nurse_service_time()
new_event=Event("departure1", time, 0)
future_event_list.append(new_event)

time=current_time+generate_nurse_service_time()
new_event=Event("departure2", time, 1)
future_event_list.append(new_event)

num_of_avai_beds-=5
healing_time=generate_hospital_healing_time()+current_time
new_event=Event("hospital", healing_time, 2)
future_event_list.append(new_event)

healing_time=generate_hospital_healing_time()+current_time
new_event=Event("hospital", healing_time, 3)
future_event_list.append(new_event)

healing_time=generate_hospital_healing_time()+current_time
new_event=Event("hospital", healing_time, 4)
future_event_list.append(new_event)

healing_time=generate_hospital_healing_time()+current_time
new_event=Event("hospital", healing_time, 5)
future_event_list.append(new_event)

healing_time=generate_hospital_healing_time()+current_time
new_event=Event("hospital", healing_time, 6)
future_event_list.append(new_event)

# arrival event of first patient is created by hand
next_arrival_time=current_time+generate_interarrival()
new_event=Event("arrival", next_arrival_time, total_arrivals)
future_event_list.append(new_event)

while (total_healed_patient) < 20:
#while (total_healed_patient) < 200:
#while (total_healed_patient) < 1000:
    advance_time()
    execute_event()


total_bed_occupation += (5-num_of_avai_beds) * (current_time - last_bed_num_change_time)

# since the time nurses have been spending on triage is calculated after finishing them at the end of 
# the simulation working time of nurses are added by hand in case they are still working
if nurse1 == False: 
    nurse1_work += current_time - nurse1_start_time
if nurse2 == False: 
    nurse2_work += current_time - nurse2_start_time

1. In order to calculate marginal probabilities of being empty for triage or beds we kept duration of them being empty during the simulation and divided them to total simulation time
1. In order to calculate rate of people that are rejected we hold a variable that is incremented once everytime a patient is rejected and divided it by the total amount of critical patients
1. In order to calculate average utilization of each nurse we hold the total time a nurse is working by adding the times they are unavailable and divided it by total simulation time by taking average of both average utilization time is found.
1. In order to calculate average number of occupied beds in the hospital we multiplied the number of beds that are available with the duration that they are available for (this is equal to the area under the graph of number of available beds over time) and divided this result to total time.
1. In order to calculate average number of patients that are treated at home we keep track of each patient that is sent home and divided it by total number of patients that are finished their triage phase.
1. In order to calculate average time a sick person gets better we keep track of each patient's arrival time and summed each patient's difference between arrival time and the time they have finished healing, then divided this sum to total number of patients that are finished healing. 

In [17]:
print("long run marginal probabilities of being empty for triage", (empty_nurse_time/current_time))
print("long run marginal probabilities of being empty for beds", (empty_bed_time/current_time))
print("joint probability of both being empty", (both_empty_time/current_time))
print("average number of people that are rejected due to bed unavailability", (rejected_patient/(accepted_patient+rejected_patient)))
#print("utilization of nurse1", (nurse1_work/current_time))
#print("utilization of nurse2", (nurse2_work/current_time))
print("average utilization of each triage nurse", ((nurse1_work+nurse2_work)/(2*current_time)))
print("average number of occupied beds in the hospital", (total_bed_occupation/current_time))
print("average number of patients that are treated at home", ((stable_patient+rejected_patient)/(stable_patient+rejected_patient+accepted_patient)))
print("average time a sick person gets better", (total_healing_time/total_healed_patient))

long run marginal probabilities of being empty for triage 0.5998091777215138
long run marginal probabilities of being empty for beds 0.39833246704233866
joint probability of both being empty 0.20764934237119928
average number of people that are rejected due to bed unavailability 0.5238095238095238
average utilization of each triage nurse 0.6179909455261134
average number of occupied beds in the hospital 4.534492041136982
average number of patients that are treated at home 0.6153846153846154
average time a sick person gets better 7.601359055050553
